## Benchmark with Correlation

The following simulations are based on the class DATA that can be found in Util.py.

The cell below performs an optimisation process using the method Benchmark_Correlation, which 

1. Computes an autocorrelation structure of the reservoir response 

2. Selects the best hyperparmaters couple $\theta$ and $\lambda$. $\theta$ is a threshold to perform feature selection from the 
autocorrelation structure. The reservoir responses that are correlated more than the selected value of $\theta$ are discarded, and the remaining responses are used for optimisation and testing. $\lambda$ is multiplicative factor of the penalty term for ridge-regression. Both these hyperparamters can help avoiding overfitting, and thus are optimised together through a grid search over the $(\theta, \lambda)$ space. 

Given the limited amount of data available. The Benchmark_Correlation method splits the dataset in the following way.

1. First, a percantage $p_{te}$ of data is selected for testing.

2. Then, the method performs ten-cross validation on the remaining data and selects the $(\theta, \lambda)$ that corresponds to the highest average performance (minimum error) on the validation sets. The testing performance is finally computed using such optimal $(\theta, \lambda)$ on the unseen test set selected in 1.

This procedure is repeated $1/p_{te}$ times with different test sets to be certain that the results were not due to a lucky split.


In the following, we repeat the procedure for different reservoir architectures.
To select a specific reservoir architecture to train and evaluate, it is sufficient to provide the files numbers to be used.
Below this is defined in the 'option' variable.

To train on a specific task, you can provide the task name to the class Data. Below, this is done in the 'Task' variable.
Valid Tasks names can be:

- Narma. Performs Narma 1 to 15 on the Mackey-Glass dataset. 
- SINE. Performs different sine waves transformations.
- MC. Computes the memory capacity of the selcted architecture from the Mackey-Glass dataset. When performing this task, use the method Measure_MC and not Benchmark_Correlation. See the notebook Metrics for an example of this.
- NL. Computes the non-linearity of the architecture considered. When performing this task, use the method Measure_NL and not Benchmark_Correlation. See the notebook Metrics for an example of this.
- Prediction_Narma. The method performs a Narma 7 on the future instances of the Mackey-Glass.
- If none of the above is pecified, the method performs future prediction on the Mackey-Glass dataset.




In [1]:
## Load the packages and define functions

import numpy as np
import os 
import matplotlib.pyplot as plt

from Util import *
import torch
import pandas as pd

import importlib

# This function loads all the data from the defined folder. 
# Datalen is the length of the data to analyse (default 250)
def load_data(folder, datalen = 250):
    array_init=False
    
    ## Loop through data files in each folder
    for file in os.listdir(folder):
        if '.npy' in file:
            
            # Load the data
            data=np.load(os.path.join(folder,file),allow_pickle=True)
            
            #Seperate output states and input fields
            states = data[:datalen,:-1]
            input_ = data[:datalen,-1].reshape(-1,1)
            
            # Initialise the array
            if array_init==False:
                data_all = states
                input_all = input_



                array_init = True
            # Concatenate the inputs and outputs
            else:
                if len(data)>(datalen-1):
                    data_all=np.concatenate([data_all[:datalen],data[:datalen]],axis=1)
                    input_all=np.concatenate([input_all[:datalen],input_[:datalen]],axis=1)
                else:
                    print('File has less data than required - ', file)
    return data_all, input_all
                    
# Define an arbitrary Mackley Glass time series
def Mackey_Def(T_data,b=0.2,a=.1,tau=17,delta=10):
    
    N_pred=20
    burn_in=20

    T_gen=2*(T_data+burn_in+N_pred)

    s=torch.zeros([T_gen])

    s[0:tau]=1.1*torch.rand([tau])+0.2

    for t in range(tau,T_gen):

        s[t]=s[t-1]+b*s[t-tau]/(1+s[t-tau]**delta)-a*s[t-1]
    
    S=s[np.arange(0,np.floor(T_gen/2))*2]
    
    Signal=S[burn_in:-N_pred]
    
    Y=torch.zeros([Signal.size()[0],N_pred])

    for n in range(0,N_pred):

        Y[:,n]=S[burn_in+n:-N_pred+n]
        
    return Signal, Y

# Load the Mackey Glass series used in the manuscript
def Mackey_Manuscript(T_data):
    MG_paper = np.load('mackey_glass_t17.npy')
    MG_actual=[]
    for i in range(T_data):
        MG_actual.append(MG_paper[2*i])
    return MG_actual

# Define the sine transformation tasks. Default period is 30
def Sine_Tasks(T_data,period = 30):
    
    x = np.arange(0,T_data)
    
    all_targets = []
    target_names = []
    
    ## Get sine variations
    sinx = np.sin(x*2*3.1415/(period))
    all_targets.append(sinx)
    target_names.append('sinx')
    sinp5x = np.sin(x*2*3.1415/(2*period))
    all_targets.append(sinp5x)
    target_names.append('sinp5x')
    sin2x = np.sin(x*2*3.1415/(0.5*period))
    all_targets.append(sin2x)
    target_names.append('sin2x')
    sin3x = np.sin(x*2*3.1415/((1/3)*period))
    all_targets.append(sin3x)
    target_names.append('sin3x')
    sin_squared = sinx**2
    all_targets.append(sin_squared)
    target_names.append('sin_squared')
    sin_cubed = sinx**3
    all_targets.append(sin_cubed)
    target_names.append('sin_cubed')
    
    #Get cos variations
    cosx = np.cos(x*2*3.1415/(period))
    all_targets.append(cosx)
    target_names.append('cosx')
    cosp5x = np.cos(x*2*3.1415/(2*period))
    all_targets.append(cosp5x)
    target_names.append('cosp5x')
    cos2x = np.cos(x*2*3.1415/(0.5*period))
    all_targets.append(cos2x)
    target_names.append('cos2x')
    cos3x= np.cos(x*2*3.1415/((1/3)*period))
    all_targets.append(cos3x)
    target_names.append('cos3x')
    
    #Get Saw variations
    sawx = signal.sawtooth(x*2*3.1415/(period))
    all_targets.append(sawx)
    target_names.append('sawx')
    saw2x = signal.sawtooth(x*2*3.1415/(0.5*period))
    all_targets.append(saw2x)
    target_names.append('saw2x')
    
    ## Scale all the above functions
    for i in range(len(all_targets)):
        all_targets[i] = (all_targets[i]-np.min(all_targets[i]))/(np.max(all_targets[i])-np.min(all_targets[i]))
    
    ## Append the tasks and names to a final list
    final_targets = []
    final_names = []
    for i in range(len(all_targets)):
        final_targets.append(all_targets[i])
        final_names.append(target_names[i])
    
    ## Add all of the base tasks previously defined
    for i in range(len(all_targets)):
        for j in range(len(all_targets)):
            if j>=i:
                new = (all_targets[i]+all_targets[j])
                new = (new-np.min(new))/(np.max(new)-np.min(new))
                final_targets.append(new)
                final_names.append(target_names[i]+'+'+target_names[j])

    ## Multiply all of the base tasks previously defined
    for i in range(len(all_targets)):
        for j in range(len(all_targets)):
            if j>=i:
                new = (all_targets[i]*all_targets[j])
                new = (new-np.min(new))/(np.max(new)-np.min(new))
                final_targets.append(new)
                final_names.append(target_names[i]+'*'+target_names[j])

    return final_targets, final_names

# Functions for defining the NARMA transform tasks.
def NARMA(Y,n_back,T_data):
                
                
    

    signal=np.copy(Y);
    Y=np.zeros([T_data,n_back])    

    start=15

    for n in range(n_back):

        Y[:,n]=Narma(signal[:T_data],n,start)
    return Y
        
def Narma(s, step, start):

    
    T=np.shape(s)[0]
    y=torch.zeros([T])
    
    alpha=0.3
    beta=0.01
    gamma=2
    delta=0.1
    
    for t in range(step,T):
        
        y[t]=alpha*y[t-1]+beta*y[t-1]*torch.sum(y[t-step:t])+gamma*s[t-step]*s[t-1]+delta
        
            
    return y


In [ ]:

import importlib
from scipy import io
from Util import *
#reload(Util)
import warnings
warnings.filterwarnings('ignore')                       
## We can ignore warnings. INases Ridge-Regression will 
## complain about matrix inversion. However, we are using different splits 
## to select best hyperparameters, where training is well-defined.


datafolder=r'Data\Mackey_Glass\Single\MS' ## The data folder to perform the predictions on. 


ths=np.array([0.999, 0.99, 0.98, 0.97, 0.96, 0.95, 0.94, 0.93, 0.92]) # Values of theta (Hyperparameter) to be optimised (grid-search, explained above).

len_data = 250
data, inputs = load_data(datafolder,len_data)
print('Data shape = ',data.shape)

targetname = 'MG_Predict' # Select NARMA / MG_predict / SINE

## Mackey-Glass future prediction
MG = Mackey_Manuscript(1000)
MG = (MG-np.min(MG))/(np.max(MG)-np.min(MG))

if targetname=='MG_Predict':
# Define the targets - MG future prediction
    delays = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]
    targets = np.zeros((len(delays),len_data))
    for i in range(len(delays)):
        targets[i] = MG[delays[i]:len_data+delays[i]]

if targetname=='NARMA':   
    # Define the targets - MG NARMA
    n_narma = 15
    targets = NARMA(MG,n_narma,len_data)
    
if targetname=='NARMA_pred':   
    # Define the targets - MG NARMA predict
    delays = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]
    targets_temp = np.zeros((len(delays),len_data))
    targets= np.zeros((len(delays),len_data))
    start=15
    for i in range(len(delays)):
        targets_temp[i] = MG[delays[i]:len_data+delays[i]]
    n_narma = 7
    for i in range(len(delays)):
        targets[i] = Narma(targets_temp[i],n_narma,start).T

if targetname =='SINE':
    targets, names = Sine_Tasks(250,30)


MSE_Tr1=[]
MSE_Val1=[]
MSE_Te1=[]

Feats1=[]
Best_hyper1=[]
Z_val1=[]
Z_te1=[]

# Percentage split for the test set
p_te=0.2
# The number of outer cross validations is 1/p_te 

## Initialise the class with the targets, data to be trialled and percentage split for train
Data=DATA_CLEAN(targets.T,data,p_te)

# Do an initial split of the training data to initialise the arrays
Data.CrossVal(3,1)

# Do an initial correlation analysis
Data.CORR()

'''
This function does the optimisation and returns the following data.
MSE_tr / MSE_val / MSE_te : a matrix with the Tr, Val and Te MSE's for each outer loop of cross validation
RM : The features which are removed for each outer validation
Best Hyper : The best theta, bets alpha and number of features remaining for each validation
Z_val / Z_te : Matrix with size (1/p_te, len_data, n_tasks, 2) containing the predictions for the validation 
                and test for every task and outer loop split
'''
MSE_tr, MSE_val, MSE_te, RM, Best_hyper, Z_val, Z_te=Data.Benchmark_Correlation(ths)

MSE_Tr1.append(MSE_tr)
MSE_Val1.append(MSE_val)
MSE_Te1.append(MSE_te)

Feats1.append(RM)
Best_hyper1.append(Best_hyper)
Z_val1.append(Z_val)
Z_te1.append(Z_te)

fname = targetname+'_all'
savdir = os.path.join(datafolder,'Feature_Selection')
if os.path.isdir(savdir) is False:
    os.mkdir(savdir)

## The results are saved in MATLAB files. To recreate the figures of the paper, refer to the analysis and figure plotting codes in the GitHub description
## The results will be saved within a new folder inside the data folder named 'Feature_Selection'

io.savemat(os.path.join(savdir,'Corr_MSETe'+fname+'.mat'),{"array": np.float32(MSE_Te1)})
io.savemat(os.path.join(savdir,'Corr_MSEVal'+fname+'.mat'),{"array": np.float32(MSE_Val1)})
io.savemat(os.path.join(savdir,'Corr_MSETr'+fname+'.mat'),{"array": np.float32(MSE_Tr1)})
io.savemat(os.path.join(savdir,'Corr_ZTe'+fname+'.mat'),{"array": np.float32(Z_te1)})
io.savemat(os.path.join(savdir,'Corr_Zval'+fname+'.mat'),{"array": np.float32(Z_val1)})
for i in range(len(Feats1)):
    io.savemat(os.path.join(savdir,str(i)+'Corr_Feats2'+fname+'.mat'),{"array": Feats1[i]})
io.savemat(os.path.join(savdir,'Corr_Best_hyper2'+fname+'.mat'),{"array": np.float32(Best_hyper1)})

        

## Evolutionary algorithm, fine-tuning feature selection

The cell below exploits again the class DATA, but to perform an evolutionary algorithm to fine-tune the feature selection.
It is necessary to run one of the cell above with the Benchmark_Correlation method before running the following.
Indeed, the evolutionary algorithm is initialised from the features found after optimisation of the hyperparameter $\theta$.
Thus, the method Evolutionary needs a starting Mask specifying which features to use at the beginning and the best hyperparameters $(\theta, \lambda)$ for each different split.
The cell automatically saves the results. Please, change the names accordingly to what you are running.



In [ ]:

index=0

MSE_trials=[]
Z_val_trials = []
Z_te_trials = []

# Get the feature mask
Mask=np.array(Feats1[index])

# Use the first feature mask
Mask2=np.copy(Mask[0,:])

# Get indicies of removed features
RM=np.where(Mask2==1)

#Initialise the data again
Data=DATA_CLEAN(targets.T,data,p_te)

# Get the best alphas
alphas=Best_hyper1[index][:,1]

# Mutation percentage a function of the number of features
p_mutation=1/np.shape(Mask)[1]
N_iteration=5

#Loop through the number of iteration
for i in range(N_iteration):
    print('Iteration', i)
    MSE, MSE_bench, Z_val, Z_te=Data.Evolutionary(Mask, p_mutation, N_iteration, alphas)

    MSE_trials.append(MSE)
    Z_val_trials.append(Z_val)
    Z_te_trials.append(Z_te)


io.savemat(os.path.join(savdir,'Evol_MSE_trials'+fname+'.mat'),{"array": np.float32(MSE_trials)})
io.savemat(os.path.join(savdir,'Evol_Z_te'+fname+'.mat'),{"array": np.float32(Z_te_trials)})
io.savemat(os.path.join(savdir,'Evol_Z_val'+fname+'.mat'),{"array": np.float32(Z_val_trials)})
io.savemat(os.path.join(savdir,'Evol_RM'+fname+'.mat'),{"array": np.float32(RM_trials)})

    
    